<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D1%83%D0%BC_%E2%84%96_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧪 **Лабораторный практикум № 2**  
# **Прогнозирование временных рядов: стратегии, валидация и модели экспоненциального сглаживания**

**Кафедра:** Кафедра анализа данных и технологий программирования  
**Дисциплина:** Машинное обучение в задачах прогнозирования  
**Уровень:** Магистратура, 2 курс  
**Преподаватель:** Арабов Муллошараф Курбонович  
**Консультации:** 21.09.2025  

---

## 🎯 1. Цели и задачи работы

### **Цель:**  
Освоить полный цикл построения прогнозных моделей для **реальных многомерных временных рядов**: от углублённой декомпозиции и инженерии признаков до выбора стратегии прогнозирования, корректной оценки качества и применения классических моделей экспоненциального сглаживания. Научиться приводить ряд к стационарному виду, включая **преобразование Бокса–Кокса**, и обоснованно выбирать оптимальную модель для заданного горизонта прогноза.

### **Задачи:**
1. Провести **углублённую декомпозицию** временного ряда (аддитивную и мультипликативную) и проанализировать остатки.
2. Выполнить **расширенный feature engineering**: временные признаки, лаги, скользящие статистики, циклические кодировки.
3. Реализовать **три стратегии многшагового прогнозирования**:
   - Рекурсивную,
   - Прямую,
   - Гибридную.
4. Применить **кросс-валидацию для временных рядов** по двум схемам:
   - Скользящее / расширяющееся окно,
   - TimeSeriesSplit (прогрессивное разбиение).
5. Привести ряд к стационарности с использованием:
   - Лог-трансформации,
   - **Преобразования Бокса–Кокса** (с автоматическим подбором параметра λ через `scipy.stats.boxcox_normmax`),
   - Дифференцирования (обычного и сезонного).
6. Построить и сравнить **модели экспоненциального сглаживания**:
   - Простое экспоненциальное сглаживание (SES),
   - Модель Хольта с аддитивным трендом,
   - Модель Хольта с мультипликативным трендом.
7. Провести **диагностику адекватности моделей**:
   - Анализ автокорреляции в остатках (**тест Льюнга–Бокса**, `statsmodels.stats.diagnostic.acorr_ljungbox`),
   - Проверка гомоскедастичности остатков (визуально и через тесты),
   - Анализ нормальности распределения ошибок (Q-Q plot, Shapiro–Wilk).
8. Оценить качество прогнозов по метрикам: **MAE, RMSE, MAPE**, включая сравнение с наивным прогнозом.
9. Разработать **веб-интерфейс** для интерактивного построения и сравнения прогнозов.
10. Сформулировать **выводы и рекомендации** по выбору модели в зависимости от характеристик ряда и горизонта прогноза.

> 💡 **Важно:** данная работа **не включает сложные ML-модели** (нейросети, градиентный бустинг). Акцент — на понимание фундаментальных принципов прогнозирования и корректной диагностики моделей.

> 📌 **Требования к объёму данных:**  
> - Обучающая выборка: **не менее 500 наблюдений**  
> - Тестовая выборка: **не менее 2 × h** и **минимум 50 точек**  
> Это обеспечивает статистическую надёжность оценок и визуализаций.

> 💡 **Реальные ряды редко бывают стационарными «из коробки». Преобразования — не опциональный шаг, а обязательная часть подготовки.**

---

## 📚 2. Теоретические предпосылки

В отличие от задач регрессии на независимых данных, прогнозирование временных рядов требует учёта **временной зависимости**, **нестационарности** и **структурных компонент** (тренд, сезонность). Классические модели, такие как экспоненциальное сглаживание, остаются мощным инструментом благодаря своей интерпретируемости и устойчивости.

### 🔑 Ключевые элементы:

- **Горизонт прогнозирования (h):** количество шагов вперёд.
- **Стационарность:** необходимое условие для многих моделей. Достигается через трансформации и дифференцирование.
- **Преобразование Бокса–Кокса:** параметрическое семейство преобразований, стабилизирующее дисперсию и приближающее распределение к нормальному. Включает логарифм как частный случай (λ → 0). Реализовано в `scipy.stats.boxcox` и `boxcox_normmax`.
- **Стратегии прогнозирования:** выбор между накоплением ошибки (рекурсивная) и независимостью прогнозов (прямая).
- **Кросс-валидация во временных рядах:** запрещено перемешивание; данные разбиваются только в хронологическом порядке (`sklearn.model_selection.TimeSeriesSplit`).
- **Экспоненциальное сглаживание:** семейство моделей, взвешивающих прошлые наблюдения с экспоненциально убывающими весами (`statsmodels.tsa.holtwinters.ExponentialSmoothing`).

---

## 🧪 3. Методика эксперимента и порядок выполнения

### **3.1. Этап 1. Углублённая декомпозиция и анализ остатков**

**Задача:** Разложить ряд из ЛР № 1 на компоненты и оценить качество декомпозиции.

#### 📌 Что нужно сделать:
- Применить **аддитивную** и **мультипликативную** декомпозицию (`seasonal_decompose`).
- Подобрать **оптимальный период сезонности** (7, 30, 365 и др.).
- Проанализировать **остатки**:
  - Проверить стационарность (ADF, KPSS),
  - Построить ACF/PACF остатков,
  - Проверить нормальность (гистограмма, Q-Q plot).
- Выбрать лучшую модель декомпозиции на основе анализа остатков.
- Визуализировать все компоненты.

> 🛠️ **Инструменты:** `statsmodels.tsa.seasonal.seasonal_decompose`

---

### **3.2. Этап 2. Расширенный feature engineering**

**Задача:** Создать информативные признаки для прогнозирования.

#### 📌 Что нужно сделать:
- Сгенерировать **временные признаки**:
  - День недели, месяц, квартал,
  - Циклические признаки через `sin/cos`.
- Создать **лаговые признаки**: `lag_1`, `lag_7`, `lag_30`.
- Рассчитать **скользящие статистики**:
  - Среднее, std, min, max по окнам 7, 30, 90.
- Добавить **признаки волатильности** (например, скользящий коэффициент вариации).
- (Опционально) добавить праздничные/событийные метки.

> 🛠️ **Инструменты:** `pandas`, `numpy`

---

### **3.3. Этап 3. Стратегии многопшагового прогнозирования**

**Задача:** Реализовать и сравнить три подхода к прогнозу на `h ≥ 7` шагов.

#### 📌 Что нужно сделать:
- **Рекурсивная стратегия**: одна модель → итеративное использование прогнозов.
- **Прямая стратегия**: отдельная модель для каждого шага `t+1, ..., t+h`.
- **Гибридная стратегия**: например, рекурсивная для ближайших шагов, прямая — для дальних.
- Сравнить по:
  - Точности (MAE, RMSE на каждом шаге),
  - Времени вычислений,
  - Накоплению ошибки.

> 💡 Используйте `sklearn` или собственные функции на основе `statsmodels`.

---

### **3.4. Этап 4. Кросс-валидация для временных рядов**

**Задача:** Оценить качество моделей без утечки будущего.

#### 📌 Что нужно сделать:
- Реализовать **скользящее окно**: фиксированная длина обучения, сдвиг по времени.
- Реализовать **расширяющееся окно**: обучение растёт со временем.
- Настроить **TimeSeriesSplit** (`sklearn.model_selection`).
- Для каждой схемы:
  - Оценить среднее качество по фолдам,
  - Проанализировать стабильность метрик во времени.

> 📊 Визуализируйте динамику ошибки по фолдам.

---

### **3.5. Этап 5. Приведение к стационарности и преобразования**

**Задача:** Стабилизировать дисперсию и устранить тренд/сезонность.

#### 📌 Что нужно сделать:
- Применить **лог-трансформацию** (если данные > 0).
- Выполнить **преобразование Бокса–Кокса**:
  - Подобрать оптимальный λ с помощью `scipy.stats.boxcox` + `boxcox_normmax`,
  - Сохранить λ для обратного преобразования.
- Выполнить **дифференцирование**:
  - 1-го порядка (для тренда),
  - Сезонное (для сезонности),
  - Комбинированное (например, `diff(1).diff(7)`).
- Проверить стационарность после каждого преобразования (ADF/KPSS).
- Выбрать **оптимальную цепочку преобразований**.

> 📌 **Обратное преобразование обязательно** для оценки в исходных единицах!

---

### **3.6. Этап 6. Модели экспоненциального сглаживания**

**Задача:** Построить и сравнить классические модели.

#### 📌 Что нужно сделать:
- **SES** (`ExponentialSmoothing` с `trend=None, seasonal=None`).
- **Хольт (аддитивный)**: `trend='add'`.
- **Хольт (мультипликативный)**: `trend='mul'` (только если данные > 0).
- Для каждой модели:
  - Включить `optimized=True`,
  - Построить прогноз на `h` шагов,
  - Рассчитать **доверительные интервалы** (если поддерживается),
  - Оценить качество на тестовой выборке.
- Сравнить с **наивным прогнозом** (например, `y[t+h] = y[t]`).
- Провести **диагностику адекватности**:
  - Тест Льюнга–Бокса на остатки,
  - График остатков vs прогнозов (гомоскедастичность),
  - Q-Q plot и Shapiro–Wilk тест для нормальности.

> 🛠️ **Инструменты:** `statsmodels.tsa.holtwinters.ExponentialSmoothing`, `scipy.stats.shapiro`

---

### **3.7. Этап 7. Веб-интерфейс для интерактивного прогнозирования**

**Задача:** Создать инструмент для визуального анализа и сравнения моделей.

#### 🎛️ Обязательные элементы интерфейса:
- Загрузка CSV/Parquet-файла
- Выбор целевой переменной и **горизонта прогноза h** (7, 30, 90 дней)
- Переключение между **аддитивной/мультипликативной декомпозицией**
- Настройка параметров преобразования (включая λ для Бокса–Кокса)
- **Визуализация прогнозов с доверительными интервалами**
- **Таблица сравнения метрик** (MAE, RMSE, MAPE) для всех моделей и стратегий
- Экспорт прогноза и параметров модели

#### 🧰 Технологический стек:
- `Streamlit` + `Plotly` (интерактив),
- `statsmodels`, `scipy`, `sklearn`.

> 🌐 Деплой на Hugging Face Spaces или Render — приветствуется.

---

### **3.8. Этап 8. Сравнительный анализ и выводы**

**Задача:** Обосновать выбор лучшей модели.

#### 📌 Что нужно сделать:
- Собрать таблицу метрик по всем моделям и стратегиям.
- Проанализировать:
  - Какая модель лучше на коротком/длинном горизонте?
  - Как влияет преобразование Бокса–Кокса на качество?
  - Насколько кросс-валидация согласуется с финальной оценкой?
- Сформулировать **практические рекомендации**.
- Написать **рефлексию**: что было сложным? Какие инсайты получены?

---

## 🔍 4. Дополнительные исследовательские задания (по желанию)

1. **Анализ чувствительности к λ** в преобразовании Бокса–Кокса.
2. **Прогнозные интервалы** и их покрытие на тестовой выборке.
3. **Ансамбли моделей** (усреднение прогнозов SES, Хольта и наивного).
4. **Детектирование структурных разрывов** и адаптация моделей.
5. **Байесовская оптимизация** гиперпараметров сглаживания.

---

## 📄 5. Требования к отчёту

Отчёт должен содержать:

1. Титульный лист (ФИО, группа, дата, подпись преподавателя)
2. Описание источника данных и структуры датасета (из ЛР № 1)
3. Результаты декомпозиции и анализа остатков
4. Примеры feature engineering с кодом
5. Сравнение трёх стратегий прогнозирования
6. Результаты кросс-валидации по двум схемам
7. Анализ эффективности преобразований (включая Бокса–Кокса)
8. Сравнение моделей экспоненциального сглаживания с метриками
9. Результаты **диагностики адекватности моделей**:  
   - p-value теста Льюнга–Бокса,  
   - Графики остатков на гомоскедастичность,  
   - Выводы о нормальности ошибок.
10. Скриншоты или ссылка на веб-интерфейс
11. Выводы и рефлексия

---

## 📊 6. Критерии оценки

| Оценка | Критерии |
|--------|---------|
| **5** | Полное выполнение всех этапов. Глубокий анализ декомпозиции, преобразований и стратегий. Корректная кросс-валидация. Реализованы все модели сглаживания + Бокс–Кокс. Диагностика моделей выполнена полностью. Работающий веб-интерфейс с обязательными элементами. Выполнены доп. задания. |
| **4** | Выполнены этапы 1–7. Есть сравнение моделей и стратегий. Веб-интерфейс базовый. Отчёт полный, но без глубокой интерпретации. |
| **3** | Реализованы основные этапы (1–4). Есть модели и оценка качества, но без комплексного сравнения и диагностики. |
| **2** | Работа не выполнена или отсутствует содержательный анализ. |

---

## 📖 7. Литература

1. Hyndman, R.J., Athanasopoulos, G. *Forecasting: Principles and Practice*. — 3rd ed., 2021. [https://otexts.com/fpp3/](https://otexts.com/fpp3/)  
2. Box, G.E.P., Jenkins, G.M. *Time Series Analysis: Forecasting and Control*. — 5th ed., 2015.  
3. Документация:
   - `scipy.stats.boxcox`, `boxcox_normmax` — [SciPy Docs](https://docs.scipy.org/doc/scipy/reference/stats.html)
   - `statsmodels.tsa.holtwinters`, `seasonal_decompose` — [Statsmodels Docs](https://www.statsmodels.org/stable/index.html)
   - `sklearn.model_selection.TimeSeriesSplit` — [Scikit-learn Docs](https://scikit-learn.org)
   - `streamlit`, `plotly` — [Streamlit](https://docs.streamlit.io/), [Plotly](https://plotly.com/python/)

> 💡 Эта работа закладывает основу для последующего изучения ARIMA, SARIMA, Prophet и нейросетевых моделей.
